In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    pipeline
)
from datasets import load_dataset, DatasetDict
import numpy as np
from evaluate import load

/home/noway/VSCodeProjects/CodeTeX/codetex-backup/codetex/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
bleu = load("bleu")

# Load the dataset directly from a CSV file
dataset = load_dataset('csv', data_files='data/dataset.csv')

# Ensure the dataset has 'code' and 'equation' columns
print("Dataset columns:", dataset['train'].column_names)

# Split the dataset into train, validation, and test sets
# First, split into train and test sets
dataset = dataset['train'].train_test_split(test_size=0.1)

# Then, split the test set equally into validation and test sets
test_valid = dataset['test'].train_test_split(test_size=0.5)

# Create a DatasetDict to hold the splits
dataset = DatasetDict({
    'train': dataset['train'],
    'validation': test_valid['train'],
    'test': test_valid['test'],
})

Generating train split: 73 examples [00:00, 14051.59 examples/s]

Dataset columns: ['code', 'equation']


In [43]:

# Initialize the tokenizer and model
model_name = "t5-small"  # You can replace this with another model like 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Tokenization function using 'text_target' for target sequences
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples['code'],
        max_length=512,
        truncation=True,
        padding=False,  # Padding is handled by the data collator
    )
    labels = tokenizer(
        text_target=examples['equation'],
        max_length=256,
        truncation=True,
        padding=False,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the tokenization function to the entire dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)



Map: 100%|██████████| 65/65 [00:00<00:00, 5475.26 examples/s]

Map: 100%|██████████| 4/4 [00:00<00:00, 614.17 examples/s]

Map: 100%|██████████| 4/4 [00:00<00:00, 618.86 examples/s]


In [44]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Post-process
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    decoded_labels = [[label] for label in decoded_labels]
    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["bleu"]}


In [45]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=10,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("t5-code-to-math")
tokenizer.save_pretrained("t5-code-to-math")

/tmp/ipykernel_142903/3171033834.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
 10%|█         | 9/90 [00:09<01:07,  1.20it/s]/home/noway/VSCodeProjects/CodeTeX/codetex/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

 10%|█         | 9/90 [00:10<01:07,  1.20it/s]

{'eval_loss': 1.665670394897461, 'eval_bleu': 0.3207024162225735, 'eval_runtime': 0.8512, 'eval_samples_per_second': 4.7, 'eval_steps_per_second': 1.175, 'epoch': 1.0}


 11%|█         | 10/90 [00:11<01:27,  1.09s/it]

{'loss': 2.0574, 'grad_norm': 7.57383394241333, 'learning_rate': 1.7777777777777777e-05, 'epoch': 1.11}


 20%|██        | 18/90 [00:19<00:55,  1.31it/s]

...

100%|██████████| 90/90 [01:28<00:00,  1.18it/s]

{'loss': 1.539, 'grad_norm': 7.847878932952881, 'learning_rate': 0.0, 'epoch': 10.0}


/home/noway/VSCodeProjects/CodeTeX/codetex/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

100%|██████████| 90/90 [01:31<00:00,  1.02s/it]


{'eval_loss': 1.1647580862045288, 'eval_bleu': 0.2458423950688514, 'eval_runtime': 1.0262, 'eval_samples_per_second': 3.898, 'eval_steps_per_second': 0.975, 'epoch': 10.0}
{'train_runtime': 91.4725, 'train_samples_per_second': 7.106, 'train_steps_per_second': 0.984, 'train_loss': 1.7097603585984973, 'epoch': 10.0}


('t5-code-to-math/tokenizer_config.json',
 't5-code-to-math/special_tokens_map.json',
 't5-code-to-math/tokenizer.json')

In [46]:
# Evaluate the model on the test set
results = trainer.evaluate(tokenized_datasets["test"])
print("Evaluation results:", results)

# Prepare the model for inference
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

/home/noway/VSCodeProjects/CodeTeX/codetex/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 129.56it/s]

Evaluation results: {'eval_loss': 1.0083293914794922, 'eval_bleu': 0.38694621926276307, 'eval_runtime': 0.9791, 'eval_samples_per_second': 4.086, 'eval_steps_per_second': 1.021, 'epoch': 10.0}


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [57]:
# Create a text generation pipeline
equation_generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU
)

In [59]:


def generate_equations(code_snippets, max_length=256, num_beams=4, early_stopping=True):
    """
    Generates equations from a list of code snippets.

    Args:
        code_snippets (list of str): List of code snippets to convert.
        max_length (int, optional): Maximum length of the generated equation. Defaults to 256.
        num_beams (int, optional): Number of beams for beam search. Defaults to 4.
        early_stopping (bool, optional): Whether to stop the beam search when at least `num_beams` sentences are finished. Defaults to True.

    Returns:
        list of str: Generated equations corresponding to the input code snippets.
    """
    try:
        if not isinstance(code_snippets, list):
            raise ValueError("Input should be a list of code snippets.")

        # Generate equations using the pipeline
        generated = equation_generator(
            code_snippets,
            max_length=max_length,
            num_beams=num_beams,
            early_stopping=early_stopping,
            truncation=True  # Ensure inputs are truncated to the model's max input length
        )

        # Extract the generated text
        equations = [item['generated_text'] for item in generated]
        return equations

    except Exception as e:
        print(f"An error occurred during equation generation: {e}")
        return []
    
code_examples = [
    "def add(a, b): return a + b",
    "def multiply(x, y):\n    return x * y",
    "def divide(numerator, denominator):\n    return numerator / denominator"
]

generated_equations = generate_equations(code_examples)
for code, eq in zip(code_examples, generated_equations):
    print(f"Code:\n{code}\nGenerated Equation: {eq}\n")

Code:
def add(a, b): return a + b
Generated Equation: def add(a, b)

Code:
def multiply(x, y):
    return x * y
Generated Equation: multiply(x, y)

Code:
def divide(numerator, denominator):
    return numerator / denominator
Generated Equation: def divide(numerator, denominator): return numerator / denominator

